# Feed Forward Neural Net For Stock Price Movements Prediction 

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
QUANDL_API_KEY = 'YOUR-QUANDL-API-KEY-HERE'

## Load Stock Price Data

In [ ]:
%%time

import datetime
from data_loading.stocks_data_loaders import get_stock_attribute_data
from data_loading.stocks_data_loaders import get_stock_data

tickers = ['WIKI/AAPL','WIKI/MSTF']

start_date_train=datetime.datetime(2013,1,1)

end_date_train=datetime.datetime(2017,12,31)

train_df=get_stock_data(tickers,start_date_train,end_date_train,QUANDL_API_KEY)

train_prices_df=get_stock_attribute_data(stocks_df=train_df,attribute='Adj. Close')

start_date_test=datetime.datetime(2018,1,1)

end_date_test=datetime.datetime(2018,12,31)

test_df=get_stock_data(tickers,start_date_test,end_date_test,QUANDL_API_KEY)

test_prices_df=get_stock_attribute_data(stocks_df=test_df,attribute='Adj. Close')

## Preprocess Prices For Training

In [ ]:
%%time

import numpy as np

from stock_movement_prediction.data_preprocessing import price_preprocessing

time_period=5

train_preprocessed_df=price_preprocessing(train_prices_df,time_period=time_period)
x_train=np.array(train_preprocessed_df['normalized_price_sequence'].tolist(),dtype=float)
y_train=np.array(train_preprocessed_df['onehot_encoded_price_movement'].tolist(),dtype=float)

test_preprocessed_df=price_preprocessing(test_prices_df,time_period=time_period)
x_test=np.array(test_preprocessed_df['normalized_price_sequence'].tolist(),dtype=float)
y_test=np.array(test_preprocessed_df['onehot_encoded_price_movement'].tolist(),dtype=float)

# Remove rows with NANs
is_nan=~np.isnan(x_train).any(axis=1)
x_train=x_train[is_nan]
y_train=y_train[is_nan]

is_nan=~np.isnan(x_test).any(axis=1)
x_test=x_test[is_nan]
y_test=y_test[is_nan]

## Train Neural Network

In [ ]:
%%time

import tensorflow as tf

from ml_models.neural_networks import create_feedworward_network_model

input_dim=x_train.shape[1]
hidden_layers_sizes=[100,100]
layers_activation='relu'
output_dim=y_train.shape[1]
output_activation='softmax'

model=create_feedworward_network_model(input_dim,
                                       hidden_layers_sizes,
                                       layers_activation,
                                       output_dim,
                                       output_activation)

optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

training = model.fit(x_train,y_train,epochs=1000,batch_size=16,validation_data=(x_test,y_test),shuffle=True)

## Performances Analysis

### Plot Training/Validation Metrics

In [ ]:
import matplotlib.pyplot as plt

fig,axs=plt.subplots(1,2,figsize=(8,2))

tags=['loss','accuracy']

for n in range(2):
    axs[n].plot(training.history[f'{tags[n]}'],label=f'training {tags[n]}')
    axs[n].plot(training.history[f'val_{tags[n]}'],label=f'testing {tags[n]}')
    axs[n].legend()
    axs[n].set_xlabel('training steps')
    axs[n].set_ylabel(f'{tags[n]}')
    axs[n].grid()
    
plt.show()

### Display Confusion Matrix

In [ ]:
#Yet to be implemented